In [1]:
# import modules
import xlsxwriter
import random
import pandas as pd
import numpy as np

In [2]:
from dataframe_excel_charting import DataFrameExcelCharting

In [3]:
# import pandas as pd
expected_metrics_by_node_df = pd.DataFrame(
                                           columns=[
                                                    "Settlement_ID",
                                                    "Network_ID",
                                                    "Node_ID",
                                                    "lat",
                                                    "lon",
                                                    "Marginal_Population_Served",
                                                    "Population_Served",
                                                    "Marginal_Total_Demand",
                                                    "Total_Demand",
                                                    "Total_Transit",
                                                    "Tower_Cost",
                                                    "Radio_Cost",
                                                    "Power_Cost",
                                                    "Hop_Count",
                                                    "Single_Site_Failure_Effect",
                                                    "Node_Utility",
                                                    ]
                                           )
expected_metrics_by_node_df.loc[len(expected_metrics_by_node_df)] = [
                                                                    "PER_5_216:1",
                                                                    "lat_-15.092711061872_lon_-73.745867783712",
                                                                     "F1",
                                                                     -15.092711061872,
                                                                     -73.745867783712,
                                                                     0.0,
                                                                     0.0,
                                                                     0.0,
                                                                     0.0,
                                                                     26608.5,
                                                                     15000.0,
                                                                     2500.0,
                                                                     10000.0,
                                                                     np.nan,
                                                                     26608.5,
                                                                     np.nan,
                                                                     ]
expected_metrics_by_node_df.loc[len(expected_metrics_by_node_df)] = [
                                                                    "PER_5_216:1",
                                                                    "lat_-15.092711061872_lon_-73.745867783712",
                                                                     "N1",
                                                                     -15.090779,
                                                                     -73.720231,
                                                                     10000.0,
                                                                     10000.0,
                                                                     17739.0,
                                                                     17739.0,
                                                                     26608.5,
                                                                     11000.0,
                                                                     57300.0,
                                                                     10000.0,
                                                                     1.0,
                                                                     26608.5,
                                                                     1.0,
                                                                     ]
expected_metrics_by_node_df.loc[len(expected_metrics_by_node_df)] = [
                                                                    "PER_5_216:1",
                                                                    "lat_-15.092711061872_lon_-73.745867783712",
                                                                    "N2",
                                                                    -15.113981,
                                                                    -73.723150,
                                                                    4000.0,
                                                                    4000.0,
                                                                    5913.0,
                                                                    5913.0,
                                                                    5913.0,
                                                                    15000.0,
                                                                    24800.0,
                                                                    10000.0,
                                                                    2.0,
                                                                    5913.0,
                                                                    1.0,
                                                                    ]
expected_metrics_by_node_df.loc[len(expected_metrics_by_node_df)] = [
                                                                    "PER_5_216:1",
                                                                    "lat_-15.092711061872_lon_-73.745867783712",
                                                                    "N3",
                                                                    -15.121024,
                                                                    -73.682895,
                                                                    2000.0,
                                                                    2000.0,
                                                                    2956.5,
                                                                    2956.5,
                                                                    2956.5,
                                                                    28000.0,
                                                                    24800.0,
                                                                    10000.0,
                                                                    2.0,
                                                                    2956.5,
                                                                    1.0,
                                                                    ]
expected_metrics_by_node_df.loc[len(expected_metrics_by_node_df)] = [
                                                                    "PER_5_121:1",
                                                                    "lat_-15.05366466651_lon_-73.770922082143",
                                                                    "F2",
                                                                    -15.05366466651,
                                                                    -73.770922082143,
                                                                    0.0,
                                                                    0.0,
                                                                    0.0,
                                                                    0.0,
                                                                    2217.375,
                                                                    28000.0,
                                                                    5000.0,
                                                                    10000.0,
                                                                    np.nan,
                                                                    2217.375,
                                                                    np.nan,
                                                                    ]
expected_metrics_by_node_df.loc[len(expected_metrics_by_node_df)] = [
                                                                    "PER_5_121:1",
                                                                    "lat_-15.05366466651_lon_-73.770922082143",
                                                                    "N4",
                                                                    -15.027052,
                                                                    -73.775133,
                                                                    1000.0,
                                                                    1000.0,
                                                                    1478.25,
                                                                    1478.25,
                                                                    1478.25,
                                                                    15000.0,
                                                                    24800.0,
                                                                    10000.0,
                                                                    1.0,
                                                                    1478.25,
                                                                    1.0,
                                                                    ]
expected_metrics_by_node_df.loc[len(expected_metrics_by_node_df)] = [
                                                                    "PER_5_121:1",
                                                                    "lat_-15.05366466651_lon_-73.770922082143",
                                                                    "N5",
                                                                    -15.006327,
                                                                    -73.759684,
                                                                    500.0,
                                                                    500.0,
                                                                    739.125,
                                                                    739.125,
                                                                    739.125,
                                                                    28000.0,
                                                                    24800.0,
                                                                    10000.0,
                                                                    1.0,
                                                                    739.125,
                                                                    1.0,
                                                                    ]

In [4]:
expected_metrics_by_node_df

,Settlement_ID,Network_ID,Node_ID,lat,lon,Marginal_Population_Served,Population_Served,Marginal_Total_Demand,Total_Demand,Total_Transit,Tower_Cost,Radio_Cost,Power_Cost,Hop_Count,Single_Site_Failure_Effect,Node_Utility
0,PER_5_216:1,lat_-15.092711061872_lon_-73.745867783712,F1,-15.092711,-73.745868,0.0,0.0,0.000,0.000,26608.500,15000.0,2500.0,10000.0,NaN,26608.500,NaN
1,PER_5_216:1,lat_-15.092711061872_lon_-73.745867783712,N1,-15.090779,-73.720231,10000.0,10000.0,17739.000,17739.000,26608.500,11000.0,57300.0,10000.0,1.0,26608.500,1.0
2,PER_5_216:1,lat_-15.092711061872_lon_-73.745867783712,N2,-15.113981,-73.723150,4000.0,4000.0,5913.000,5913.000,5913.000,15000.0,24800.0,10000.0,2.0,5913.000,1.0
3,PER_5_216:1,lat_-15.092711061872_lon_-73.745867783712,N3,-15.121024,-73.682895,2000.0,2000.0,2956.500,2956.500,2956.500,28000.0,24800.0,10000.0,2.0,2956.500,1.0
4,PER_5_121:1,lat_-15.05366466651_lon_-73.770922082143,F2,-15.053665,-73.770922,0.0,0.0,0.000,0.000,2217.375,28000.0,5000.0,10000.0,NaN,2217.375,NaN
5,PER_5_121:1,lat_-15.05366466651_lon_-73.770922082143,N4,-15.027052,-73.775133,1000.0,1000.0,1478.250,1478.250,1478.250,15000.0,24800.0,10000.0,1.0,1478.250,1.0
6,PER_5_121:1,lat_-15.05366466651_lon_-73.770922082143,N5,-15.006327,-73.759684,500.0,500.0,739.125,739.125,739.125,28000.0,24800.0,10000.0,1.0,739.125,1.0


In [9]:
workbook = xlsxwriter.Workbook("test_charting.xlsx")
test_class = DataFrameExcelCharting(expected_metrics_by_node_df, workbook)
test_class.getTopN(["Marginal_Population_Served", "Marginal_Total_Demand", "Total_Demand"])
test_class.writeToExcel("test_sheet")
test_class.topNChart(columns=["Marginal_Population_Served", "Marginal_Total_Demand", "Total_Demand"], n=10, category_col="Node_ID")
test_class.bucketsNChart(column="Marginal_Total_Demand",n_buckets=3)
print test_class.getBucketsCounts(column="Total_Transit", n_buckets=5)
# test_class.scatterPlot(["col1", "col4"], "category", "scatter")
# test_class.insertImage("A", test_class.num_rows + 3 + test_class._num_charts * 15, "test_geoplot.png")
test_class.geoPlot(text_col="Node_ID", value_col="Total_Transit", lat="lat", lon="lon", 
                   n_buckets=5, image_name="geo_plot", 
                   scale=100, plot_type="scattergeo", 
                scope='south america', map_type="natural earth")
test_class.closeWorkBook()

Dataframe has been changed, please write to Excel again
(array([4, 1, 0, 0, 2]), ['[739.0, 5913.0)', '[5913.0, 11087.0)', '[11087.0, 16261.0)', '[16261.0, 21435.0)', '[21435.0, 26609.0)'])


In [21]:
import plotly
import pandas as pd

# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_us_cities.csv')
df = expected_metrics_by_node_df
df.head()

df['text'] = df['Network_ID'] + df["Node_ID"]
limits = [(0,5000),(5001,10000),(10001,30000)]
colors = ['rgb(128.0, 64.0, 64.0)', 'rgb(115.0, 128.0, 64.0)', 'rgb(64.0, 128.0, 89.0)']
cities = []
scale = 100

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[(df["Single_Site_Failure_Effect"] >= lim[0]) & (df["Single_Site_Failure_Effect"] <= lim[1])]
    city = dict(
        type = 'scattergeo',
        #locations = ["peru"],
        #locationmode = "country names",
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['Single_Site_Failure_Effect'] / scale,
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    cities.append(city)

layout = dict(
        title = 'Single Site Failure Effect',
        showlegend = True,
        geo = dict(
            scope='south america',
            projection=dict( type='equirectangular', scale = 1),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
# py.iplot(fig, validate=False, filename='d3-bubble-map-populations' )
plotly.offline.plot(fig, validate=False, filename='peru-location.html')

'file:///Users/arthur/Documents/tensorflow/dataframe_excel_charting/src/peru-location.html'

In [22]:
df[(df["Single_Site_Failure_Effect"] >= 0) & (df["Single_Site_Failure_Effect"] <= 10000)]

,Settlement_ID,Network_ID,Node_ID,lat,lon,Marginal_Population_Served,Population_Served,Marginal_Total_Demand,Total_Demand,Total_Transit,Tower_Cost,Radio_Cost,Power_Cost,Hop_Count,Single_Site_Failure_Effect,Node_Utility,text
2,PER_5_216:1,lat_-15.092711061872_lon_-73.745867783712,N2,-15.113981,-73.723150,4000.0,4000.0,5913.000,5913.000,5913.000,15000.0,24800.0,10000.0,2.0,5913.000,1.0,lat_-15.092711061872_lon_-73.745867783712N2
3,PER_5_216:1,lat_-15.092711061872_lon_-73.745867783712,N3,-15.121024,-73.682895,2000.0,2000.0,2956.500,2956.500,2956.500,28000.0,24800.0,10000.0,2.0,2956.500,1.0,lat_-15.092711061872_lon_-73.745867783712N3
4,PER_5_121:1,lat_-15.05366466651_lon_-73.770922082143,F2,-15.053665,-73.770922,0.0,0.0,0.000,0.000,2217.375,28000.0,5000.0,10000.0,NaN,2217.375,NaN,lat_-15.05366466651_lon_-73.770922082143F2
5,PER_5_121:1,lat_-15.05366466651_lon_-73.770922082143,N4,-15.027052,-73.775133,1000.0,1000.0,1478.250,1478.250,1478.250,15000.0,24800.0,10000.0,1.0,1478.250,1.0,lat_-15.05366466651_lon_-73.770922082143N4
6,PER_5_121:1,lat_-15.05366466651_lon_-73.770922082143,N5,-15.006327,-73.759684,500.0,500.0,739.125,739.125,739.125,28000.0,24800.0,10000.0,1.0,739.125,1.0,lat_-15.05366466651_lon_-73.770922082143N5


In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_us_cities.csv')

In [8]:
workbook = xlsxwriter.Workbook("geo_plot.xlsx")
test_class = DataFrameExcelCharting(df, workbook)
counts, limits = test_class.getBucketsCounts("pop", 5, False)
colors = test_class._getRGBColors(5)

In [10]:
for i in range(len(limits)):
    lim = limits[i]
    df_sub = test_class.data[(test_class.data["pop"] >= lim[0]) & (test_class.data["pop"] < lim[1])]
    print df_sub.head()

            name      pop        lat         lon
4  Philadelphia   1539313  39.952335  -75.163789
5       Phoenix   1465114  33.446768 -112.075672
6   San Antonio   1359174  29.424600  -98.495141
7     San Diego   1321016  32.717421 -117.162771
8        Dallas   1219399  32.776196  -96.796899
       name      pop        lat        lon
2  Chicago   2705627  41.875555 -87.624421
3  Houston   2129784  29.758938 -95.367697
           name      pop        lat         lon
1  Los Angeles   3826423  34.053717 -118.242727
Empty DataFrame
Columns: [name, pop, lat, lon]
Index: []
Empty DataFrame
Columns: [name, pop, lat, lon]
Index: []


In [16]:
from plotly.offline import plot
import plotly.graph_objs as go

# plot([go.Scatter(x=[1, 2, 3], y=[3, 2, 6])], filename='my-graph.html')
# We can also download an image of the plot by setting the image parameter
# to the image format we want
test = plot([go.Scatter(x=[1, 2, 3], y=[3, 2, 6])], filename='my-graph', image='svg', auto_open=False)

In [14]:
print test

file:///Users/arthur/Documents/tensorflow/dataframe_excel_charting/src/my-graph.html


In [8]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my app")
location = geolocator.reverse("48.8588443, 2.2943506")

print(location.address)

GeocoderInsufficientPrivileges: HTTP Error 403: Forbidden

In [22]:
from geopy.geocoders import Nominatim
import urllib2

def get_geolocator():
    geolocator = Nominatim()

    requester = geolocator.urlopen

    def requester_hack(req, **kwargs):
        req = urllib2.Request(url=req, headers=geolocator.headers)
        return requester(req, **kwargs)

    geolocator.urlopen = requester_hack

    return geolocator

location = get_geolocator().reverse("-15.113981, -73.723150")
# print location.continent

In [41]:
print location.raw["address"]["country_code"].upper()

PE


In [35]:
from country

u'Per\xfa'